In [5]:
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy.orm import Session
from sqlalchemy import MetaData
from sqlalchemy import ForeignKey
from sqlalchemy.orm import DeclarativeBase
from sqlalchemy import insert
from sqlalchemy import select
from typing import List
from typing import Optional
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.orm import relationship
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer,Float, String, MetaData
from sqlalchemy.orm import Session


In [6]:
engine = create_engine("sqlite+pysqlite:///C://Study//CSE 4000//New folder//Thesis//dataset//New folder//weather.sf.db", echo=False)


In [7]:
# Load data using Pandas (replace 'your_data.csv' with your dataset file)
data = pd.read_csv('C://Study//CSE 4000//New folder//Thesis//dataset//New folder//300weather_data_sf_2004_2023.csv')

# Get the column names and data types from the CSV
column_names = data.columns
column_data_types = {col: data[col].dtype for col in column_names}

# Create a new MetaData object and a users table based on the column names and data types
metadata = MetaData()

WeatherSFTable = Table(
    'WeatherSFTable',
    metadata,
    Column('id', Integer, primary_key=True),
    *[Column(name, String) if dtype == 'object' else
      Column(name, Integer) if dtype == 'int64' else
      Column(name, Float) for name, dtype in column_data_types.items()]
)

# Create the table in the database
metadata.create_all(engine)

# Start a session
session = Session(engine)

# Loop through the data and insert into the table, skipping rows with null values
for _, row in data.iterrows():
    datarow = {name: row[name] for name in column_names if not pd.isna(row[name])}
    session.execute(WeatherSFTable.insert().values(datarow))
    session.commit()


In [8]:
# SELECT the first few rows with column names
query = WeatherSFTable.select().limit(5)  # You can adjust the limit as needed
result = session.execute(query)

# Get the column names and the data
column_names = result.keys()
data = result.fetchall()

# Print the column names
print(column_names)

# Print the first few rows of data
for row in data:
    print(row)

# Close the session
#session.close()

# Commit changes
session.commit()

RMKeyView(['id', 'Unnamed: 0', 'expire_time_gmt', 'valid_time_gmt', 'temp', 'wx_phrase', 'heat_index', 'rh', 'pressure', 'vis', 'wc', 'wdir', 'wspd', 'max_temp', 'min_temp', 'expire_date_time', 'valid_date_time'])
(1, 0, 1072950960, 1072943760, 12.0, 'Cloudy', 12.0, 80.0, 1013.64, 16.0, 12.0, 160.0, 22.0, 12.0, 7.0, '01/01/2004 16:00:00', '01/01/2004 14:00:00')
(2, 1, 1072954560, 1072947360, 11.0, 'Cloudy', 11.0, 83.0, 1013.34, 12.0, 11.0, 150.0, 22.0, None, None, '01/01/2004 17:00:00', '01/01/2004 15:00:00')
(3, 2, 1072958160, 1072950960, 11.0, 'Cloudy', 11.0, 89.0, 1012.84, 8.0, 11.0, 120.0, 20.0, None, None, '01/01/2004 18:00:00', '01/01/2004 16:00:00')
(4, 3, 1072958640, 1072951440, 11.0, 'Cloudy', 11.0, 88.0, 1012.85, 6.0, 11.0, 120.0, 24.0, None, None, '01/01/2004 18:00:00', '01/01/2004 16:00:00')
(5, 4, 1072961760, 1072954560, 11.0, 'Cloudy / Windy', 11.0, 86.0, 1012.94, 8.0, 11.0, 170.0, 33.0, None, None, '01/01/2004 19:00:00', '01/01/2004 17:00:00')
